In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [13]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

warnings.filterwarnings('ignore')

datasets_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/datasets/'
pred_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/Predict/'
sub_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/Predict/sub/'

# 模型融合

In [31]:
## 模型stacking 
from sklearn import linear_model # 用简单的线性模型进行stacking
from sklearn.model_selection import RepeatedKFold
# 1914-1941三个模型的预测文件
pred_xgb_A = pd.read_csv(pred_path + 'per_store_pred(xgb)_A.csv')
pred_lgbm_A = pd.read_csv(pred_path + 'per_store_pred(lgb)_A.csv')
pred_cb_A = pd.read_csv(pred_path + 'per_store_pred(cb)_A.csv')
# 1942-1969三个模型的预测文件
pred_xgb_B = pd.read_csv(pred_path + 'per_store_pred(xgb)_B.csv')
pred_lgbm_B = pd.read_csv(pred_path + 'per_store_pred(lgb)_B.csv')
pred_cb_B = pd.read_csv(pred_path + 'per_store_pred(cb)_B.csv')
# 读取1914-1941的CA1真值
true_value = pd.read_csv(datasets_path + 'sales_train_evaluation.csv') # 用evaluation的真值训练得到的线性模型，来预测B榜3个模型28天的融合值

col = [f'd_{i}' for i in range(1914, 1942)]
pred = [f'F{i}' for i in range(1, 29)]
stack_predict = pd.DataFrame() # 空的dataframe用于存放新的预测值
B_predict = pd.DataFrame() 
stack_predict = pred_lgbm_A[['id']]
B_predict = pred_lgbm_A[['id']]

for day in range(1,29):
  print(f'预测第{day}次')
  xgb_A = pred_xgb_A[[f'F{day}']]
  xgb_B = pred_xgb_B[[f'F{day}']]
  lgbm_A = pred_lgbm_A[[f'F{day}']]
  lgbm_B = pred_lgbm_B[[f'F{day}']]
  cb_A = pred_cb_A[[f'F{day}']]
  cb_B = pred_cb_B[[f'F{day}']]

  true = true_value[col[day-1]]

  stack = np.hstack([xgb_A, lgbm_A, cb_A])  # h：horizontal 水平的
  test_stack = np.hstack([xgb_B, lgbm_B, cb_B]) # B榜预测值，从1942开始一直到1969

  folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=2020) # 5折CV两次 5*2=10

  oof_stack = np.zeros(stack.shape[0]) # 30490
  predictions = np.zeros(test_stack.shape[0])

  for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(stack, true)): 
    # print("fold {}".format(fold_))
    trn_data, trn_y = stack[trn_idx], true[trn_idx]
    val_data, val_y = stack[val_idx], true[val_idx]
    Reg = linear_model.BayesianRidge()
    Reg.fit(trn_data, trn_y)
    oof_stack[val_idx] = Reg.predict(val_data)
    predictions += Reg.predict(test_stack) / 10

  stack_predict[pred[day-1]] = oof_stack
  B_predict[pred[day-1]] = predictions

# print(stack_predict) # 用于wrmesse的本地验证
# print(B_predict) # 作为B榜的提交
B_predict.to_csv(pred_path + 'Stacking.csv',index = False)
# 用于生成提交预测的文件
submission = pd.read_csv(datasets_path + 'sample_submission.csv')[['id']] 
submission = submission.merge(B_predict, on=['id'], how='left').fillna(0)
submission.to_csv(sub_path + 'submission_by_Stacking.csv', index=False) 

预测第1次
预测第2次
预测第3次
预测第4次
预测第5次
预测第6次
预测第7次
预测第8次
预测第9次
预测第10次
预测第11次
预测第12次
预测第13次
预测第14次
预测第15次
预测第16次
预测第17次
预测第18次
预测第19次
预测第20次
预测第21次
预测第22次
预测第23次
预测第24次
预测第25次
预测第26次
预测第27次
预测第28次


# 本地验证

In [ ]:
My_test = pd.read_csv(base_m5 + 'My_pre.csv') # 保存的时候，没有设置index = false，所以读取的时候有一个索引列，列名为 "Unnamed: 0"
My_test_28 = My_test[['id'] + [f'F{i}' for i in range(1,29)]] # 等价于My_test_28 = My_test.drop(columns = 'Unnamed: 0')

store = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']

My_test_CA1 = My_test_28[My_test_28.id.str.endswith('CA_1_validation')] 
My_test_CA2 = My_test_28[My_test_28.id.str.endswith('CA_2_validation')]
My_test_CA3 = My_test_28[My_test_28.id.str.endswith('CA_3_validation')]
My_test_CA4 = My_test_28[My_test_28.id.str.endswith('CA_4_validation')]
My_test_TX1 = My_test_28[My_test_28.id.str.endswith('TX_1_validation')]
My_test_TX2 = My_test_28[My_test_28.id.str.endswith('TX_2_validation')]
My_test_TX3 = My_test_28[My_test_28.id.str.endswith('TX_3_validation')]
My_test_WI1 = My_test_28[My_test_28.id.str.endswith('WI_1_validation')]
My_test_WI2 = My_test_28[My_test_28.id.str.endswith('WI_2_validation')]
My_test_WI3 = My_test_28[My_test_28.id.str.endswith('WI_3_validation')]

Compute_rate_CA1 = My_test_CA1.copy()
Compute_rate_CA2 = My_test_CA2.copy()
Compute_rate_CA3 = My_test_CA3.copy()
Compute_rate_CA4 = My_test_CA4.copy()
Compute_rate_TX1 = My_test_TX1.copy()
Compute_rate_TX2 = My_test_TX2.copy()
Compute_rate_TX3 = My_test_TX3.copy()
Compute_rate_WI1 = My_test_WI1.copy()
Compute_rate_WI2 = My_test_WI2.copy()
Compute_rate_WI3 = My_test_WI3.copy()

# 以上这些商店对应的索引不发生改变
all_preds1 = pd.concat([Compute_rate_CA1, Compute_rate_CA2, Compute_rate_CA3, Compute_rate_CA4,
             Compute_rate_TX1, Compute_rate_TX2, Compute_rate_TX3, Compute_rate_WI1,
             Compute_rate_WI2, Compute_rate_WI3],axis=0).reset_index(drop = True) # reset_index(drop = True)加上这个就不用考虑索引问题了
all_preds1

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.841742,0.757436,0.728389,0.761665,1.000136,1.165551,1.208051,0.863785,0.957409,0.854942,0.791007,0.895135,1.075710,0.912531,0.849813,0.865339,0.797540,0.737261,0.867501,1.077549,1.094876,0.867376,0.856909,0.832007,0.831587,0.986544,1.106234,1.074874
1,HOBBIES_1_002_CA_1_validation,0.203211,0.189806,0.173399,0.199822,0.230208,0.313719,0.312664,0.251374,0.229473,0.189454,0.183435,0.220875,0.266777,0.252461,0.236547,0.203273,0.205744,0.213234,0.227807,0.287858,0.289828,0.218054,0.202842,0.193763,0.194609,0.207611,0.297041,0.277692
2,HOBBIES_1_003_CA_1_validation,0.454542,0.445043,0.445441,0.441526,0.577491,0.763762,0.703281,0.472421,0.453774,0.482746,0.452602,0.537261,0.711261,0.587494,0.439546,0.441157,0.389276,0.436310,0.528160,0.681939,0.653393,0.475834,0.434501,0.428781,0.449918,0.568355,0.686396,0.651168
3,HOBBIES_1_004_CA_1_validation,1.585677,1.316821,1.288205,1.562391,1.993648,3.072055,3.293523,1.607361,1.322533,1.328573,1.584307,1.706179,2.863095,2.423045,1.571796,1.453240,1.351362,1.423681,1.923382,2.477563,3.110499,1.612639,1.451468,1.308784,1.395521,1.843039,2.854923,3.324280
4,HOBBIES_1_005_CA_1_validation,0.880313,0.749764,0.771451,0.849284,1.119681,1.400065,1.621155,0.945945,0.923281,0.983670,0.897931,1.002446,1.462214,1.067597,0.938336,0.962994,0.848742,0.914662,1.034251,1.449809,1.501207,0.963205,0.830177,0.876086,0.870730,1.103279,1.547502,1.531163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0.343249,0.337044,0.312435,0.305590,0.369307,0.439830,0.390787,0.440919,0.429687,0.325614,0.441604,0.429158,0.526028,0.581849,0.475181,0.375862,0.475694,0.458055,0.448120,0.542798,0.605173,0.451962,0.401201,0.353336,0.340917,0.379655,0.427658,0.452884
30486,FOODS_3_824_WI_3_validation,0.304118,0.271227,0.249701,0.248959,0.296205,0.351925,0.316012,0.395658,0.392394,0.292009,0.343717,0.354665,0.416399,0.420076,0.410827,0.334240,0.369710,0.364893,0.333732,0.493928,0.525711,0.361954,0.291856,0.256410,0.241357,0.239221,0.324765,0.326271
30487,FOODS_3_825_WI_3_validation,0.634179,0.526229,0.479149,0.489881,0.620834,0.704025,0.807710,1.195724,1.164279,0.783981,1.082302,1.254445,1.167874,1.212325,1.310124,0.806256,1.155137,1.193137,0.890896,1.283364,1.512414,1.007699,0.734683,0.753961,0.666271,0.680109,0.855007,0.770190
30488,FOODS_3_826_WI_3_validation,0.918011,0.905366,0.792682,0.828906,0.983269,1.187842,1.036656,1.132715,1.153527,0.863996,1.200988,1.077779,1.245104,1.204511,1.174640,0.916391,1.027401,1.083202,0.961620,1.354991,1.374730,0.974573,0.873504,0.830928,0.777575,0.926294,1.076613,1.154267


In [ ]:
################################## 本地wrmse #############################################################
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import gc

# 加载前面预先计算好的各个权重
file_pass = '/content/drive/Shared drives/MyShare/M5/Metric/'
sw_df = pd.read_pickle(file_pass+'sw_df.pkl')
S = sw_df.s.values
W = sw_df.w.values
SW = sw_df.sw.values
roll_mat_df = pd.read_pickle(file_pass+'roll_mat_df.pkl')
roll_index = roll_mat_df.index
roll_mat_csr = csr_matrix(roll_mat_df.values)


def rollup(v):
    return (v.T*roll_mat_csr.T).T

# 计算 WRMSSE 评估指标
def wrmsse(preds, y_true, score_only=False, s = S, w = W, sw=SW):
    '''
    preds - Predictions: pd.DataFrame of size (30490 rows, N day columns)
    y_true - True values: pd.DataFrame of size (30490 rows, N day columns)
    sequence_length - np.array of size (42840,)
    sales_weight - sales weights based on last 28 days: np.array (42840,)
    '''
    
    if score_only:
        return np.sum(
                np.sqrt(
                    np.mean(
                        np.square(rollup(preds.values-y_true.values))
                            ,axis=1)) * sw )*(1/12)
    else: 
        score_matrix = (np.square(rollup(preds.values-y_true.values)) * np.square(w)[:, None])  / s[:, None]
        score = np.sum(np.sqrt(np.mean(score_matrix,axis=1)))*(1/12)
        return score, score_matrix

In [ ]:
## 预测值
cols = [f'F{i}' for i in range(1,29)]
pred = all_preds1.copy()
pred = pred[cols] # 没有id列，用作wrmsse的参数

## 真值
true = pd.read_csv(base_m5 + 'sales_train_evaluation.csv') 
col2 = [f'd_{i}' for i in range(1914, 1942)]
true_lable = true.copy()
true_lable_1 = true_lable[col2]

## 进行测试
wrmsse(true_lable_1, pred, score_only=True) 
# 0.48996653939892076  0.47343329889704044

0.47343329889704044